In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score
import pandas as pd
import shutil
import os
import numpy as np

def copy_models_to_kaggle(model_paths):
    os.makedirs('/kaggle/working/models', exist_ok=True)
    new_paths = []
    for model_path in model_paths:
        model_trained_data_size = model_path.split("/")[3]
        model_trained_img_size = model_path.split("/")[5]
        new_model_path = '/kaggle/working/models/' + model_trained_data_size + "_dataset" + "_imgsize_" + model_trained_img_size + "_model.keras"
        new_paths.append(new_model_path)
        if os.path.exists(new_model_path):
            # raise Exception(f"model already exists at {new_model_path}")
            print(f"model already exists at {new_model_path}")
        shutil.copy(model_path, new_model_path)
    return new_paths
 

batch_size = 32
test_dataset_paths = [
    # '/kaggle/input/sample-1800',
    # '/kaggle/input/sample-2500',
    # '/kaggle/input/clean-2500-letters-only',
    # '/kaggle/input/clean-1800-letters-only'
    '/kaggle/input/2500-baa-taa-thaa',
    '/kaggle/input/1800-baa-taa-thaa'
]
models_paths = [
        # '/kaggle/input/1800-model/keras/16/1/model img_size_16.keras',
        # '/kaggle/input/1800-model/keras/32/1/model img_size_32.keras',
        # '/kaggle/input/1800-model/keras/64/1/model img_size_64.keras',
        # '/kaggle/input/1800-model/keras/128/1/model img_size_128.keras',
        # '/kaggle/input/1800-model/keras/256/1/model img_size_256.keras',
        # '/kaggle/input/1800-model/keras/300/1/model img_size_300.keras',

        # '/kaggle/input/2500-model/keras/16/1/model img_size_16.keras',
        # '/kaggle/input/2500-model/keras/32/1/model img_size_32.keras',
        # '/kaggle/input/2500-model/keras/64/1/model img_size_64.keras',
        # '/kaggle/input/2500-model/keras/128/1/model img_size_128.keras',
        # '/kaggle/input/2500-model/keras/256/1/model img_size_256.keras',
        # '/kaggle/input/2500-model/keras/300/1/model img_size_300.keras'
        '/kaggle/input/1800-baa-taa-thaa-model/keras/imgsize_128/1/model_5616_images_img_size_128.keras'
]
print("copying models to kaggle to be able to work with WRITE ACESS ...")
copied_models_paths = copy_models_to_kaggle(models_paths)
print(f"loading models in RAM...")
model_path_to_model = {}
for model_path in copied_models_paths:
    print(f"loading model in : {model_path} ...")
    model = load_model(model_path)
    model_path_to_model[model_path] = model

all_datasets_accuracies = {}
for dataset_path in test_dataset_paths:
    dataset_name  = dataset_path.split("/")[-1]
    print(f"Testing dataset : {dataset_path} ...")
    dataset_test_accuracies = {}
    for model_path in copied_models_paths:
        # /kaggle/working/models/2500-model_dataset_imgsize_300_model.keras
        trained_data_size = int(model_path.split("/")[4].split("-")[0])
        print(f"trained data size : {trained_data_size}")
        img_size = int(model_path.split("/")[4].split("_")[-2])
        print(f"image size : {img_size}")
        current_model_name =  str(trained_data_size) + " ," + str(img_size) + "_model"
        print(f"using : {current_model_name} ...")
        model = model_path_to_model[model_path]

        print(f"Testing model : {current_model_name} ...")
        test_datagen = ImageDataGenerator(rescale=1./255)
        test_generator = test_datagen.flow_from_directory(
            dataset_path,
            target_size=(img_size, img_size),
            batch_size=batch_size,
            class_mode='categorical',
            shuffle=False)
        print()

        # Predict the output
        steps = int(np.ceil(test_generator.samples / test_generator.batch_size))
        predictions = model.predict(test_generator, steps=steps)
        predictions = predictions.argmax(axis=-1)                    
        true_classes = test_generator.classes
        accuracy = round(accuracy_score(true_classes, predictions), 3)
        dataset_test_accuracies[current_model_name] = accuracy
        print(f"Accuracy of model {current_model_name} on dataset {dataset_name} : {accuracy}")
    all_datasets_accuracies[dataset_name] = dataset_test_accuracies

print(f"all datasets accuracies : {all_datasets_accuracies}")
accuracies_df = pd.DataFrame(all_datasets_accuracies).transpose()
accuracies_df = accuracies_df.style.background_gradient(cmap='tab20_r').set_table_styles([dict(selector="th", props=[("text-align", "center")])])
display(accuracies_df)


2024-04-22 15:17:01.080666: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-22 15:17:01.080791: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-22 15:17:01.194170: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


copying models to kaggle to be able to work with WRITE ACESS ...
loading models in RAM...
loading model in : /kaggle/working/models/1800-model_dataset_imgsize_16_model.keras ...
loading model in : /kaggle/working/models/1800-model_dataset_imgsize_32_model.keras ...
loading model in : /kaggle/working/models/1800-model_dataset_imgsize_64_model.keras ...
loading model in : /kaggle/working/models/1800-model_dataset_imgsize_128_model.keras ...
loading model in : /kaggle/working/models/1800-model_dataset_imgsize_256_model.keras ...
loading model in : /kaggle/working/models/1800-model_dataset_imgsize_300_model.keras ...
loading model in : /kaggle/working/models/2500-model_dataset_imgsize_16_model.keras ...
loading model in : /kaggle/working/models/2500-model_dataset_imgsize_32_model.keras ...
loading model in : /kaggle/working/models/2500-model_dataset_imgsize_64_model.keras ...
loading model in : /kaggle/working/models/2500-model_dataset_imgsize_128_model.keras ...
loading model in : /kaggle

/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 157ms/step
Accuracy of model 1800 ,16_model on dataset sample-1800 : 0.174
trained data size : 1800
image size : 32
using : 1800 ,32_model ...
Testing model : 1800 ,32_model ...
Found 448 images belonging to 28 classes.



/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


...

/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1442/1442 ━━━━━━━━━━━━━━━━━━━━ 95s 66ms/step
Accuracy of model 2500 ,256_model on dataset clean-1800-letters-only : 0.043
trained data size : 2500
image size : 300
using : 2500 ,300_model ...
Testing model : 2500 ,300_model ...
Found 46130 images belonging to 28 classes.

   1/1442 ━━━━━━━━━━━━━━━━━━━━ 3:11 133ms/step

/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1442/1442 ━━━━━━━━━━━━━━━━━━━━ 130s 90ms/step
Accuracy of model 2500 ,300_model on dataset clean-1800-letters-only : 0.049
all datasets accuracies : {'sample-1800': {'1800 ,16_model': 0.174, '1800 ,32_model': 0.4, '1800 ,64_model': 0.475, '1800 ,128_model': 0.833, '1800 ,256_model': 0.058, '1800 ,300_model': 0.04, '2500 ,16_model': 0.069, '2500 ,32_model': 0.08, '2500 ,64_model': 0.036, '2500 ,128_model': 0.04, '2500 ,256_model': 0.056, '2500 ,300_model': 0.049}, 'sample-2500': {'1800 ,16_model': 0.036, '1800 ,32_model': 0.042, '1800 ,64_model': 0.037, '1800 ,128_model': 0.024, '1800 ,256_model': 0.06, '1800 ,300_model': 0.036, '2500 ,16_model': 0.929, '2500 ,32_model': 0.988, '2500 ,64_model': 0.996, '2500 ,128_model': 0.979, '2500 ,256_model': 0.99, '2500 ,300_model': 0.975}, 'clean-2500-letters-only': {'1800 ,16_model': 0.036, '1800 ,32_model': 0.043, '1800 ,64_model': 0.036, '1800 ,128_model': 0.027, '1800 ,256_model': 0.058, '1800 ,300_model': 0.036, '2500 ,16_model': 0.933, '2500

,"1800 ,16_model","1800 ,32_model","1800 ,64_model","1800 ,128_model","1800 ,256_model","1800 ,300_model","2500 ,16_model","2500 ,32_model","2500 ,64_model","2500 ,128_model","2500 ,256_model","2500 ,300_model"
sample-1800,0.174000,0.400000,0.475000,0.833000,0.058000,0.040000,0.069000,0.080000,0.036000,0.040000,0.056000,0.049000
sample-2500,0.036000,0.042000,0.037000,0.024000,0.060000,0.036000,0.929000,0.988000,0.996000,0.979000,0.990000,0.975000
clean-2500-letters-only,0.036000,0.043000,0.036000,0.027000,0.058000,0.036000,0.933000,0.984000,0.998000,0.980000,0.986000,0.975000
clean-1800-letters-only,0.156000,0.404000,0.465000,0.842000,0.060000,0.040000,0.067000,0.073000,0.040000,0.041000,0.043000,0.049000
